# Setup and download the models

In [21]:
import os, sys
import urllib.request as req
import certifi
try:
    import pycurl
except ImportError:
    raise
    print("pycurl is not installed or is not found by python")
    exit()

if not os.path.exists("input_data"):
    os.makedirs("input_data")

glove_data_dir = "glove_data"
if not os.path.exists(glove_data_dir):
    os.makedirs(glove_data_dir)

if os.path.exists("data"):
    print("Deleted previous output data")
    !rm -rf data
    
if not os.path.isfile("input_data/stanfordSentimentTreebank.zip"):
    with open("input_data/stanfordSentimentTreebank.zip", 'wb') as f:
        c = pycurl.Curl() 
        c.setopt(c.URL, "https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip")
        c.setopt(c.WRITEDATA, f)
        c.setopt(c.CAINFO, certifi.where())
        c.perform()
        c.close()
    
if not os.path.isfile("input_data/stanfordSentimentTreebankRaw.zip"):
    with open("input_data/stanfordSentimentTreebankRaw.zip", 'wb') as j:
        c = pycurl.Curl()
        c.setopt(c.URL, "https://nlp.stanford.edu/~socherr/stanfordSentimentTreebankRaw.zip")
        c.setopt(c.WRITEDATA, j)
        c.setopt(c.CAINFO, certifi.where())
        c.perform()
        c.close()
        
if not os.path.isfile("input_data/trainDevTestTrees_PTB.zip"):
    with open("input_data/trainDevTestTrees_PTB.zip", 'wb') as h:
        c = pycurl.Curl()
        c.setopt(c.URL, "https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip")
        c.setopt(c.WRITEDATA, h)
        c.setopt(c.CAINFO, certifi.where())
        c.perform()
        c.close()
        
if not os.path.isfile("input_data/glove.840B.300d.zip"):
    with open("input_data/glove.840B.300d.zip", 'wb') as h:
        c = pycurl.Curl()
        c.setopt(c.URL, "https://nlp.stanford.edu/data/glove.840B.300d.zip")
        c.setopt(c.WRITEDATA, h)
        c.setopt(c.CAINFO, certifi.where())
        c.perform()
        c.close()
        
        
import zipfile
with zipfile.ZipFile("input_data/stanfordSentimentTreebank.zip", 'r') as zip_ref:
    zip_ref.extractall("input_data/")
zip_ref.close()
with zipfile.ZipFile("input_data/stanfordSentimentTreebankRaw.zip", 'r') as zip_ref:
    zip_ref.extractall("input_data/")
zip_ref.close()
with zipfile.ZipFile("input_data/trainDevTestTrees_PTB.zip", 'r') as zip_ref:
    zip_ref.extractall("input_data/")
zip_ref.close()
if not os.path.isfile(glove_data_dir + "/glove.840B.300d.txt"):
    with zipfile.ZipFile("input_data/glove.840B.300d.zip", 'r') as zip_ref:
        zip_ref.extractall(glove_data_dir)
    zip_ref.close()

print("Notebook is running at: "+ str(os.getcwd()))
print(sys.path)
import platform;

print(platform.sys.version);

Notebook is running at: /home/gfair/pnnl-xai/Code/GabrielFair/SST_LSTM
['/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/home/gfair/.ipython']
3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]


In [22]:
try:
    import numpy as np
except ImportError:
    print("numpy is not installed or is not found by python")

# try:
#     from theano import tensor as T
#     from theano import config
# except ImportError:
#     print("theano is not installed or is not found by python")
    
try:
    import tensorflow as tf
#     sess = tf.InteractiveSession()
except ImportError:
    print("tensorflow is not installed or is not found by python")
    
#try:
#    import tensorflow_fold as td
#except ImportError:
#    print("tensorflow_fold is not installed or is not found by python:")
#    print("https://github.com/tensorflow/fold/blob/master/tensorflow_fold/g3doc/setup.md")

try:
    import lasagne
except ImportError:
    print("lasagne is not installed or is not found by python")

import pickle
import codecs

try:
    import torch
except ImportError:
    print("torch is not installed or is not found by python")
    
try:
    import sklearn
except ImportError:
    print("sklearn is not installed or is not found by python")
    
try:
    import scipy
except ImportError:
    print("scipy is not installed or is not found by python")
    
try:
    from nltk.tokenize import sexpr
except ImportError:
    print("nltk is not installed or is not found by python")


lasagne is not installed or is not found by python


# Import the data

In [23]:
def load_trees(filename):
    with codecs.open(filename, encoding='utf-8') as f:
        # Drop the trailing newline and strip \s.
        trees = [line.strip().replace('\\', '') for line in f]
        print('loaded %s trees from %s' % (len(trees), filename))
        return trees


vocab = set()
with codecs.open("input_data/stanfordSentimentTreebank/SOStr.txt", encoding='utf-8') as f:
    for line in f:
        # Drop the trailing newline and strip backslashes. Split into words.
        vocab.update(line.strip().replace('\\', '').split('|'))

def check_if_word_embeddings_exist():
    if not os.path.isfile(glove_data_dir + '/filtered_glove.txt'):
        print('Saving glove files to %s' % glove_data_dir)
        nread = 0
        nwrote = 0
        with codecs.open(glove_data_dir + "/glove.840B.300d.txt", encoding='utf-8') as f:
            with codecs.open(glove_data_dir + '/filtered_glove.txt', 'w', encoding='utf-8') as out:
                for line in f:
                    nread += 1
                    line = line.strip()
                    if not line: continue
                    if line.split(u' ', 1)[0] in vocab:
                        out.write(line + '\n')
                        nwrote += 1
        print('read %s lines, wrote %s' % (nread, nwrote))
    return


check_if_word_embeddings_exist()
train_trees = load_trees("input_data/trees/train.txt")
dev_trees = load_trees("input_data/trees/dev.txt")
test_trees = load_trees("input_data/trees/test.txt")

loaded 8544 trees from input_data/trees/train.txt
loaded 1101 trees from input_data/trees/dev.txt
loaded 2210 trees from input_data/trees/test.txt


# Utilities to Tokenize and load the Stanford corpus

In [24]:
def tokenize(s):
    label, phrase = s[1:-1].split(None, 1)
    return label, sexpr.sexpr_tokenize(phrase)

def load_embeddings(embedding_path):
    """Loads embedings, returns weight matrix and dict from words to indices."""
    print('loading word embeddings from %s' % embedding_path)
    weight_vectors = []
    word_idx = {}
    with codecs.open(embedding_path, encoding='utf-8') as f:
        for line in f:
            word, vec = line.split(u' ', 1)
            word_idx[word] = len(weight_vectors)
            weight_vectors.append(np.array(vec.split(), dtype=np.float32))
    # Annoying implementation detail; '(' and ')' are replaced by '-LRB-' and
    # '-RRB-' respectively in the parse-trees.
    word_idx[u'-LRB-'] = word_idx.pop(u'(')
    word_idx[u'-RRB-'] = word_idx.pop(u')')
    # Random embedding vector for unknown words.
    weight_vectors.append(np.random.uniform(
        -0.05, 0.05, weight_vectors[0].shape).astype(np.float32))
    return np.stack(weight_vectors), word_idx

weight_matrix, word_idx = load_embeddings(glove_data_dir + '/filtered_glove.txt')

loading word embeddings from glove_data/filtered_glove.txt


# Class to hold the LSTM Object

In [25]:
class TreeLSTMModel(object):
    def __init__(self, weight_matrix, word_idx, ModelConfig):

        self.ModelConfig = ModelConfig

        self.word_embedding = td.Embedding(*weight_matrix.shape, initializer=weight_matrix, name='word_embedding')
        self.word_idx = word_idx

        self.keep_prob_ph = tf.placeholder_with_default(1.0, [])
        self.tree_lstm = td.ScopedLayer(
            tf.contrib.rnn.DropoutWrapper(
                BinaryTreeLSTMCell(self.ModelConfig.lstm_num_units, keep_prob=self.keep_prob_ph),
                input_keep_prob=self.keep_prob_ph, output_keep_prob=self.keep_prob_ph),
            name_or_scope='tree_lstm')
        self.output_layer = td.FC(self.ModelConfig.num_classes, activation=None, name='output_layer')

        self.embed_subtree = td.ForwardDeclaration(name='embed_subtree')
        self.model = self.embed_tree(is_root=True)
        self.embed_subtree.resolve_to(self.embed_tree(is_root=False))

        self.compiler = td.Compiler.create(self.model)
        print('input type: %s' % self.model.input_type)
        print('output type: %s' % self.model.output_type)

        self.metrics = {k: tf.reduce_mean(v) for k, v in self.compiler.metric_tensors.items()}

        self.loss = tf.reduce_sum(self.compiler.metric_tensors['all_loss'])
        opt = tf.train.AdagradOptimizer(ModelConfig.learning_rate)

        grads_and_vars = opt.compute_gradients(self.loss)
        found = 0
        for i, (grad, var) in enumerate(grads_and_vars):
            if var == self.word_embedding.weights:
                found += 1
                grad = tf.scalar_mul(ModelConfig.embedding_learning_rate, grad)
                grads_and_vars[i] = (grad, var)
        assert found == 1  # internal consistency check
        self.train_op = opt.apply_gradients(grads_and_vars)

        self.saver = tf.train.Saver()
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def train_epoch(self, train_set, batch_size):
        loss = 0
        for batch in td.group_by_batches(train_set, batch_size):
            train_feed_dict = {self.keep_prob_ph: self.ModelConfig.keep_prob, self.compiler.loom_input_tensor: batch}
            loss += self.train_step(train_feed_dict)
        return loss

    def train_step(self, train_feed_dict):
        _, batch_loss = self.sess.run([self.train_op, self.loss], train_feed_dict)
        return batch_loss

    def dev_eval(self, epoch, train_loss, dev_feed_dict):
        dev_metrics = self.sess.run(self.metrics, dev_feed_dict)
        dev_loss = dev_metrics['all_loss']
        dev_accuracy = ['%s: %.2f' % (k, v * 100) for k, v in
                        sorted(dev_metrics.items()) if k.endswith('hits')]
        print('epoch:%4d, train_loss: %.3e, dev_loss_avg: %.3e, dev_accuracy:\n  [%s]'
              % (epoch, train_loss, dev_loss, ' '.join(dev_accuracy)))
        return dev_metrics['root_hits']

    def embed_tree(self, is_root):
        """Creates a block that embeds trees; output is tree LSTM state."""
        return td.InputTransform(tokenize) >> td.OneOf(
            key_fn=lambda pair: pair[0] == '2',  # label 2 means neutral
            case_blocks=(self.add_metrics(is_root, is_neutral=False),
                         self.add_metrics(is_root, is_neutral=True)),
            pre_block=(td.Scalar('int32'), self.logits_and_state()))

    def logits_and_state(self):
        """Creates a block that goes from tokens to (logits, state) tuples."""
        unknown_idx = len(self.word_idx)
        lookup_word = lambda word: self.word_idx.get(word, unknown_idx)

        word2vec = (td.GetItem(0) >> td.InputTransform(lookup_word) >>
                    td.Scalar('int32') >> self.word_embedding)

        pair2vec = (self.embed_subtree(), self.embed_subtree())

        # Trees are binary, so the tree layer takes two states as its input_state.
        zero_state = td.Zeros((self.tree_lstm.state_size,) * 2)
        # Input is a word vector.
        zero_inp = td.Zeros(self.word_embedding.output_type.shape[0])

        word_case = td.AllOf(word2vec, zero_state)
        pair_case = td.AllOf(zero_inp, pair2vec)

        tree2vec = td.OneOf(len, [(1, word_case), (2, pair_case)])

        return tree2vec >> self.tree_lstm >> (self.output_layer, td.Identity())

    def add_metrics(self, is_root, is_neutral):
        """A block that adds metrics for loss and hits; output is the LSTM state."""
        c = td.Composition(
            name='predict(is_root=%s, is_neutral=%s)' % (is_root, is_neutral))
        with c.scope():
            # destructure the input; (labels, (logits, state))
            labels = c.input[0]
            logits = td.GetItem(0).reads(c.input[1])
            state = td.GetItem(1).reads(c.input[1])

            # calculate loss
            loss = td.Function(self.tf_node_loss)
            td.Metric('all_loss').reads(loss.reads(logits, labels))
            if is_root:
                td.Metric('root_loss').reads(loss)

            # calculate fine-grained hits
            hits = td.Function(self.tf_fine_grained_hits)
            td.Metric('all_hits').reads(hits.reads(logits, labels))
            if is_root:
                td.Metric('root_hits').reads(hits)

            # calculate binary hits, if the label is not neutral
            if not is_neutral:
                binary_hits = td.Function(self.tf_binary_hits).reads(logits, labels)
                td.Metric('all_binary_hits').reads(binary_hits)
                if is_root:
                    td.Metric('root_binary_hits').reads(binary_hits)

            # output the state, which will be read by our by parent's LSTM cell
            c.output.reads(state)
        return c

    @staticmethod
    def tf_node_loss(logits, labels):
        return tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)

    @staticmethod
    def tf_fine_grained_hits(logits, labels):
        predictions = tf.cast(tf.argmax(logits, 1), tf.int32)
        return tf.cast(tf.equal(predictions, labels), tf.float64)

    @staticmethod
    def tf_binary_hits(logits, labels):
        softmax = tf.nn.softmax(logits)
        binary_predictions = (softmax[:, 3] + softmax[:, 4]) > (softmax[:, 0] + softmax[:, 1])
        binary_labels = labels > 2
        return tf.cast(tf.equal(binary_predictions, binary_labels), tf.float64)

# Train the LSTM model after tokenizing it

In [26]:
# Tai et al. https://arxiv.org/pdf/1503.00075.pdf

class ModelConfig(object):
    num_classes = 5
    lstm_num_units = 300  # Tai et al. used 150, but our regularization strategy is more effective
    learning_rate = 0.05
    keep_prob = 0.75
    batch_size = 100
    epochs = 20
    embedding_learning_rate = 0.1
    
tree_lstm_model = TreeLSTMModel(weight_matrix, word_idx, ModelConfig)
train_set = tree_lstm_model.compiler.build_loom_inputs(train_trees)

NameError: name 'td' is not defined

In [ ]:
dev_feed_dict = tree_lstm_model.compiler.build_feed_dict(dev_trees)

best_accuracy = 0.0
save_path = os.path.join(PathConfig.data_path, 'sentiment_model')
for epoch, shuffled in enumerate(td.epochs(train_set, ModelConfig.epochs), 1):
    train_loss = tree_lstm_model.train_epoch(shuffled, ModelConfig.batch_size)
    accuracy = tree_lstm_model.dev_eval(epoch, train_loss, dev_feed_dict)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        checkpoint_path = tree_lstm_model.saver.save(tree_lstm_model.sess, save_path, global_step=epoch)
        print('model saved in file: %s' % checkpoint_path)